# Trabalho 2 IART: Aprendizagem Supervisionada - Regressão
## Introdução
O futebol é possivelmente o desporto mais popular de sempre com milhões de adeptos e um igual número de jogadores dado ser talvez um dos desportos mais fáceis de jogar que existe.

Para além disso, este desporto movimenta uma grande quantidade de dinheiro quer pela venda de bilhetes e _merchandise_ quer pelas transferências de jogadores. Outro lado desta economia é o mercado das apostas desportivas que é cada vez mais procurado por adeptos. Sabe-se que os apostadores profissionais acertam na sua aposta cerca de 53% das vezes.

Sendo assim, o objetivo deste trabalho é prever o resultado de jogos de maneira a criar um sistema que recomenda o resultado do jogo em que apostar, preferivelmente com uma precisão superior à dos apostadores profissionais.

## Descrição do _dataset_
O _dataset_ inclui informação acerca de 25000 partidas de futebol em ligas de 11 países europeus realizadas nas épocas de 2008 a 2016 e acerca de 10000 jogadores. Este inclui ainda informação sobre os _odds_ fornecidos por vários _sites_ de apostas para os vários jogos.
Mais informação pode ser encontrada [aqui](https://www.kaggle.com/hugomathien/soccer).

## Preparação dos dados
O _dataset_ que irá ser usado corresponde a uma base de dados sqlite que inclui as tabelas a seguir apresentadas e que contêm a seguinte informação:

In [9]:
import sqlite3

conn = sqlite3.connect('soccer.db')
cursor = conn.cursor()
cursor.execute("select name from sqlite_master where type='table'; ")
cursor.fetchall()

[('sqlite_sequence',),
 ('Player_Attributes',),
 ('Player',),
 ('Match',),
 ('League',),
 ('Country',),
 ('Team',),
 ('Team_Attributes',)]

* **Country:**  Países para os quais existe informação.
* **League:** Ligas de futebol dos vários países incluídos na tabela _Country_.
* **Match:** Informação sobre as várias partidas de futebol que inclui estatísticas do jogo, _odds_, bem como os jogadores que participaram.
* **Player:** Informação sobre os vários jogadores da base de dados, incluindo nome, data de nascimento, altura e peso.
* **Player_Attributes:** Habilidades de cada um dos jogadores tiradas do jogo FIFA.
* **Team:** Informação acerca de cada uma das equipas, incluindo o seu nome.
* **Team_Attributes:** Habilidades de cada uma das equipas tiradas do jogo FIFA.

Dado tratar-se de um _dataset_ extenso (~300MB) apenas serão consideradas partidas jogadas na English Premier League, já que corresponde a uma liga com um número razoável de equipas e jogos. Sendo assim, os passos seguintes têm como objetivo extrair informação acerca desta liga.

In [25]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("soccer.db")

# countries = pd.read_sql_query('select * from Country', conn)
# countries.head()

# leagues = pd.read_sql_query('select * from League', conn)
# leagues.head()

matches = pd.read_sql_query("select League.name, Match.season, T1.team_long_name as home_team_name, T2.team_long_name as away_team_name, stage, date, B365H, B365D, B365A from League, Match on (League.id = Match.league_id), Team as T1 on (T1.team_api_id = Match.home_team_api_id), Team as T2 on (T2.team_api_id = Match.away_team_api_id) where League.name = 'England Premier League'", conn)
matches.head()

# teams = pd.read_sql_query('select * from Team', conn)
# teams.head()

,name,season,home_team_name,away_team_name,stage,date,B365H,B365D,B365A
0,England Premier League,2008/2009,Manchester United,Newcastle United,1,2008-08-17 00:00:00,1.29,5.5,11.00
1,England Premier League,2008/2009,Arsenal,West Bromwich Albion,1,2008-08-16 00:00:00,1.20,6.5,15.00
2,England Premier League,2008/2009,Sunderland,Liverpool,1,2008-08-16 00:00:00,5.50,3.6,1.67
3,England Premier League,2008/2009,West Ham United,Wigan Athletic,1,2008-08-16 00:00:00,1.91,3.4,4.20
4,England Premier League,2008/2009,Aston Villa,Manchester City,1,2008-08-17 00:00:00,1.91,3.4,4.33


In [11]:
matches = pd.read_sql_query('select home_player_X1 from Match', conn)
matches.loc[matches["home_player_X1"].isnull()]

,home_player_X1
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
24732,NaN
24733,NaN
24734,NaN
24735,NaN


In [23]:
# rascunho
# obter informação sobre jogos especificos de um dado país incluindo o nome
# dos clubes envolvidos e as odds do jogo
df = pd.read_sql_query("select T1.team_long_name, T2.team_long_name, Country.name, date, BSH, BSD, BSA from Country, Match inner join Team as T1 on (Match.home_team_api_id = T1.team_api_id) inner join Team as T2 on (Match.away_team_api_id = T2.team_api_id) where Match.country_id = 17642 and Country.id = Match.country_id;", conn)
df.head()

,team_long_name,team_long_name,name,date,BSH,BSD,BSA
0,FC Porto,CF Os Belenenses,Portugal,2008-08-24 00:00:00,1.33,4.0,9.00
1,Sporting CP,Trofense,Portugal,2008-08-23 00:00:00,1.20,5.0,12.00
2,Vitória Guimarães,Vitória Setúbal,Portugal,2008-08-22 00:00:00,1.91,3.1,3.75
3,FC Paços de Ferreira,SC Braga,Portugal,2008-08-23 00:00:00,2.88,3.0,2.30
4,Amadora,Académica de Coimbra,Portugal,2008-08-24 00:00:00,2.10,3.0,3.25
